In [26]:
import pandas as pd
import datetime
import time
import dateutil.parser
import re

import selenium # pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from msedge.selenium_tools import EdgeOptions, Edge # pip install msedge-selenium-tools selenium==3.141

from os import remove

def limite_posts():
    """Se ingresa la fecha hasta la que se buscan posts"""
    
    while True:
        try:
            fecha_limite = input("Ingrese la fecha inicial en formato YYYY-MM-DD: ")
            fecha_limite = datetime.datetime.strptime(fecha_limite, '%Y-%m-%d')
            break
        except:
            print("El formato debe ser YYYY-MM-DD")
    fecha_limite = fecha_limite.strftime('%Y-%m-%d')
    fecha_limite = int(time.mktime(datetime.datetime.strptime(fecha_limite, '%Y-%m-%d').timetuple()))
    return fecha_limite

def iniciar_driver():
    """ Crea una instancia del WebDriver
        El Web Driver debe estar en la misma ruta que el script
    """
    options = EdgeOptions()
    options.use_chromium = True
    # options.add_argument("headless")
    # prefs = {"profile.managed_default_content_settings.images": 2}
    # options.add_experimental_option("prefs", prefs)
    driver = Edge(options = options)
    return driver

def crear_dataframe(data):
    data_frame = pd.DataFrame(data,
                      columns=['type', 'id', 'create_time', 'user', 'url_user', 'comment', 'url_comment', 'likes'],
                      index=None)
    return data_frame

def exportar(data):
    try:
        remove("instagramComments.xlsx")
        data.to_excel("instagramComments.xlsx")
    except:
        data.to_excel("instagramComments.xlsx")

def extraer_informacion_respuestas(contenedor_reply):
    """Extrae la información de cada respuesta"""
    
    # tipo
    tipo = "reply"
    print(tipo)
            
    # id respuesta
    try:
        id_respuesta = contenedor_reply.find_element_by_xpath('.//a[@class="gU-I7"]').get_attribute("href")
        id_respuesta = re.search('/r/(.+?)/', id_respuesta).group(1)
    except Exception as ex:
        print(ex)
        id_respuesta = ""
    print(id_respuesta)
    
    # Fecha
    try:
        create_time = datetime.datetime.strftime(dateutil.parser.parse(
            contenedor_reply.find_element_by_xpath('.//time').get_attribute('datetime')), '%d/%m/%Y %H:%M:%S')
    except Exception as ex:
        print(ex)
        create_time = ""
    print(create_time)
    
    # usuario
    try:
        user = contenedor_reply.find_element_by_css_selector('a.sqdOP.yWX7d._8A5w5.ZIAjV').text
    except Exception as ex:
        print(ex)
        user = ""
    print(user)
    
    # url usuario
    try:
        url_user = contenedor_reply.find_element_by_css_selector('a.sqdOP.yWX7d._8A5w5.ZIAjV').get_attribute("href")
    except Exception as ex:
        print(ex)
        url_user = ""
    print(url_user)
    
    # contenido de la respuesta
    try:
        reply = contenedor_reply.find_element_by_xpath('.//div[@class="C4VMK"]/span').text.replace('\n', ' ')
    except Exception as ex:
        print(ex)
        reply = ""
    print(reply)
    
    # url respuesta
    try:
        url_reply = contenedor_reply.find_element_by_xpath('.//a[@class="gU-I7"]').get_attribute("href")
    except Exception as ex:
        print(ex)
        url_reply = ""
    print(url_reply)
    
    # likes
    try:
        likes = contenedor.find_element_by_xpath('.//*[contains(text(), "Me gusta")]').text.replace('Me gusta', '')
    except Exception as ex:
        likes = "0"
    print(likes)
    
    respuesta = (tipo, id_respuesta, create_time, user, url_user, reply, url_reply, likes)
    
    
    return respuesta

def extraer_informacion_comentarios(contenedor):
    """Extrae la información de cada comentario"""
    
    global comments_instagram
    
    # Tipo
    tipo = "comment"
    print(tipo)
    
    # Id comentario
    try:
        id_comentario = contenedor.find_element_by_xpath('.//a[@class="gU-I7"]').get_attribute("href")
        id_comentario = re.search('/c/(.+?)/', id_comentario).group(1)
    except Exception as ex:
        print(ex)
        id_comentario = ""
    print(id_comentario)
    
    # Fecha
    try:
        create_time = datetime.datetime.strftime(dateutil.parser.parse(
            contenedor.find_element_by_xpath('.//time').get_attribute('datetime')), '%d/%m/%Y %H:%M:%S')
    except Exception as ex:
        print(ex)
        create_time = ""
    print(create_time)
    
    # usuario
    try:
        user = contenedor.find_element_by_css_selector('a.sqdOP.yWX7d._8A5w5.ZIAjV').text
    except Exception as ex:
        print(ex)
        user = ""
    print(user)
    
    # url usuario
    try:
        url_user = contenedor.find_element_by_css_selector('a.sqdOP.yWX7d._8A5w5.ZIAjV').get_attribute("href")
    except Exception as ex:
        print(ex)
        url_user = ""
    print(url_user)
    
    # comentario
    try:
        comment = contenedor.find_element_by_xpath('.//div[@class="C4VMK"]/span').text.replace('\n', ' ')
    except Exception as ex:
        print(ex)
        comment = ""
    print(comment)
    
    # url comentario
    try:
        url_comment = contenedor.find_element_by_xpath('.//a[@class="gU-I7"]').get_attribute("href")
    except Exception as ex:
        print(ex)
        url_comment = ""
    print(url_comment)
    
    # likes
    try:
        likes = contenedor.find_element_by_xpath('.//*[contains(text(), "Me gusta")]').text.replace('Me gusta', '')
    except Exception as ex:
        likes = "0"
    print(likes)
    
    comentario = (tipo, id_comentario, create_time, user, url_user, comment, url_comment, likes)
    comments_instagram.append(comentario)
    
    # Obtener respuestas a comentarios
    try:
        contenedores_respuestas = contenedor.find_elements_by_css_selector('li.gElp9.rUo9f.lDe-V')
    except Exception as ex:
        pass
    
    if contenedores_respuestas:
        for contenedor_reply in contenedores_respuestas:
            respuesta = extraer_informacion_respuestas(contenedor_reply)
            comments_instagram.append(respuesta)

def obtener_contenedores_comentarios(driver):
    """Obtiene los contenedores de cada comentario en el post de instagram"""
    
    global control_fecha
    
    time.sleep(1)
    
    try:        
        fecha_post = datetime.datetime.strftime(
            dateutil.parser.parse(driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]'
                                                          ).get_attribute("datetime")), '%d/%m/%Y %H:%M:%S')
    except:
        try:
            fecha_post = datetime.datetime.strftime(
                dateutil.parser.parse(driver.find_element_by_css_selector('time._1o9PC.Nzb55'
                                                              ).get_attribute("datetime")), '%d/%m/%Y %H:%M:%S')
        except:
            try:
                fecha_post = datetime.datetime.strftime(
                    dateutil.parser.parse(
                        driver.find_element_by_xpath(
                            '/html/body/div[1]/div/div/section/main/div/div[1]/article/div/div[2]/div/div[2]/div[2]/a/time'
                        ).get_attribute("datetime")), '%d/%m/%Y %H:%M:%S')
            except:
                return
    control_fecha = int(time.mktime(datetime.datetime.strptime(fecha_post, '%d/%m/%Y %H:%M:%S').timetuple()))
    
    time.sleep(1)
    try:
        focus = driver.find_element_by_xpath('//ul[@class="XQXOT pXf-y"]/div')
    except:
        try:
            focus = driver.find_element_by_xpath('//ul[@class="XQXOT    pXf-y "]/div')
        except:
            try:
                focus = driver.find_element_by_css_selector('ul.XQXOT')
            except:
                return
    try:
        focus.send_keys(Keys.NULL)
    except:
        try:
            driver.execute_script("arguments[0].focus();", focus)
        except:
            return
    
    while True:
        try:
            time.sleep(0.8)
            more_comments_btn = driver.find_element_by_css_selector('body > div._2dDPU.QPGbb.CkGkG > div._32yJO > div > article > div > div.HP0qD > div > div > div.eo2As > div.EtaWk > ul > li > div > button > div > svg')
            time.sleep(0.1)
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", more_comments_btn)
            time.sleep(0.1)
            more_comments_btn.click()
        except Exception as ex:
            break
    
    try:
        time.sleep(0.5)
        answers = driver.find_elements_by_xpath('//span[contains(text(), "Ver respuestas")]')
        time.sleep(0.1)
        for node in answers:
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", node)
            time.sleep(0.5)
            node.click()
    except Exception as ex:
        pass
    
    try:
        time.sleep(0.7)
        contenedores_comentarios = driver.find_elements_by_css_selector('ul.Mr508')
        for contenedor in contenedores_comentarios:
            comentario = extraer_informacion_comentarios(contenedor)
    except Exception as ex:
        print(ex)
        
def obtener_posts(contenedor, driver):
    """Obtiene los posts de instagram que se encuentran dentro del contenedor y se abre cada post con el fin de extraer
       los comentarios
    """
    
    posts = contenedor.find_elements_by_xpath('.//div[@class="v1Nh3 kIKUG  _bz0w"]')
    for post in posts:
        # post.click()
        time.sleep(2)
        link = post.find_element_by_xpath('./a').get_attribute("href")
        driver.execute_script('window.open("' + link + '");')
        time.sleep(0.5)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(2)
        obtener_contenedores_comentarios(driver)
        time.sleep(0.5)
        # driver.find_element_by_xpath('//body').send_keys(Keys.ESCAPE)
        # time.sleep(0.5)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        # Verifíca la fecha límite
        if control_fecha < fecha_limite:
            return
        
def obtener_grupo_contenedores(driver):
    """Obtiene los contenedores donde se encuentran los posts de instagram"""
    
    last_height = driver.execute_script("return window.pageYOffset;")
    scrolling = True
    id_contenedores = set()
    
    global fecha_limite
    global control_fecha
    
    while scrolling:
        grupo_contenedor = driver.find_elements_by_xpath('//div[@class="Nnq7C weEfm"]')
        for contenedor in grupo_contenedor:
            id_contenedor = str(contenedor)
            if id_contenedor not in id_contenedores:
                id_contenedores.add(id_contenedor)
                driver.execute_script("arguments[0].scrollIntoView({block: 'center', behavior: 'smooth'});", contenedor)
                time.sleep(0.5)
                obtener_posts(contenedor, driver)
                # Verifíca la fecha límite
                if control_fecha < fecha_limite:
                    driver.close()
                    return
                time.sleep(2)
    driver.close()
    
def validar_sesion(driver):
    time.sleep(5)
    try:
        element = driver.find_element_by_xpath('//input[@name="username"]')
    except:
        return False
    return True

fecha_limite = limite_posts()
control_fecha = ""

comments_instagram = []
    
url = "https://www.instagram.com"
url_unisinu = "https://www.instagram.com/unisinuoficial"
    
driver = iniciar_driver()
driver.get(url)
    
espera = True
    
while espera:
    print("Iniciando sesión...")
    espera = validar_sesion(driver)
    if espera == False:
        print("¡Sesión iniciada!")
        break
driver.get(url_unisinu)
time.sleep(1)
obtener_grupo_contenedores(driver)

print("Completado")

Ingrese la fecha inicial en formato YYYY-MM-DD: 2021-01-01
Iniciando sesión...
Iniciando sesión...
Iniciando sesión...
Iniciando sesión...
Iniciando sesión...
Iniciando sesión...
¡Sesión iniciada!
comment
18155725081175986
19/11/2021 00:42:56
13marlis
https://www.instagram.com/13marlis/
#YoquieroaUnisinu97 👏👏👏
https://www.instagram.com/p/CWb977_rDLA/c/18155725081175986/
0
comment
17877329669594065
19/11/2021 00:48:32
amador.karo
https://www.instagram.com/amador.karo/
👏#yoQuieroaUnisinu31
https://www.instagram.com/p/CWb977_rDLA/c/17877329669594065/
0
comment
17912870729162297
19/11/2021 01:06:06
leila_naranjo
https://www.instagram.com/leila_naranjo/
👏
https://www.instagram.com/p/CWb977_rDLA/c/17912870729162297/
0
comment
18152281534200727
19/11/2021 01:51:36
keila_vergaraa
https://www.instagram.com/keila_vergaraa/
#YoQuieroAUnisinu98
https://www.instagram.com/p/CWb977_rDLA/c/18152281534200727/
0
comment
17898428138420919
19/11/2021 02:05:38
brayan.vinasco
https://www.instagram.com/braya

2 
comment
18194578132121502
13/11/2021 01:36:45
aichelbaquero
https://www.instagram.com/aichelbaquero/
❤️🔥
https://www.instagram.com/p/CWMQTZfrqgP/c/18194578132121502/
2 
comment
17919359054020150
13/11/2021 16:16:23
nicoleale26
https://www.instagram.com/nicoleale26/
👏👏
https://www.instagram.com/p/CWMQTZfrqgP/c/17919359054020150/
2 
comment
17908095503203638
14/11/2021 00:26:29
soranajv16
https://www.instagram.com/soranajv16/
Feliz de haber sido parte del programa y ganadora de uno de ellos. 👏🙌 #YoQuieroAUnisinu72
https://www.instagram.com/p/CWMQTZfrqgP/c/17908095503203638/
1 
comment
17935453075779074
16/11/2021 15:02:59
keila_vergaraa
https://www.instagram.com/keila_vergaraa/
#YoQuieroAUnisinu98
https://www.instagram.com/p/CWMQTZfrqgP/c/17935453075779074/
0
comment
17918296352046140
16/11/2021 15:08:22
yairaterris
https://www.instagram.com/yairaterris/
#YoQuieroaUnisinu37
https://www.instagram.com/p/CWMQTZfrqgP/c/17918296352046140/
0
comment
17896901369320303
16/11/2021 19:25:17
13m

1 
comment
17929505821863831
10/11/2021 23:06:22
julieth_conde61
https://www.instagram.com/julieth_conde61/
Felicidades por esa gran visita #YoquieroaUnisinu43
https://www.instagram.com/p/CWHL0JmLVWu/c/17929505821863831/
1 
comment
17941609822641591
10/11/2021 22:48:10
vergarateff
https://www.instagram.com/vergarateff/
Un proyecto más de calidad ❤️🔥 #YoQuieroaUnisinu8
https://www.instagram.com/p/CWHL0JmLVWu/c/17941609822641591/
2 
comment
17930113852855008
11/11/2021 00:03:17
andrex.perez
https://www.instagram.com/andrex.perez/
#YoQuieroaUnisinu16 Sede de nuevos acuerdos de prosperidad 👏
https://www.instagram.com/p/CWHL0JmLVWu/c/17930113852855008/
1 
comment
17933174815794985
11/11/2021 13:48:23
88.vanessaa.88
https://www.instagram.com/88.vanessaa.88/
👏👏 #yoquieroUnisinu15
https://www.instagram.com/p/CWHL0JmLVWu/c/17933174815794985/
1 
comment
17896494986425279
12/11/2021 21:50:41
sofia.chaves10
https://www.instagram.com/sofia.chaves10/
Uno de los mejores espacios dentro de la campaña.

17925318256912231
10/11/2021 16:39:29
esp_feder
https://www.instagram.com/esp_feder/
Excelente noticia 🙌🔥 #YoQuieroaUnisinu51
https://www.instagram.com/p/CWCaRLirTX0/c/17925318256912231/
1 
comment
17870190491555511
10/11/2021 18:29:07
juan_camilo_chica_v18
https://www.instagram.com/juan_camilo_chica_v18/
#yoquieroaunisinu
https://www.instagram.com/p/CWCaRLirTX0/c/17870190491555511/
1 
comment
17898952091497703
10/11/2021 02:16:17
ange1mar
https://www.instagram.com/ange1mar/
Felicidades #YoQuieroaUnisinu19
https://www.instagram.com/p/CWCaRLirTX0/c/17898952091497703/
1 
comment
17939126581728764
10/11/2021 18:38:50
julieth_conde61
https://www.instagram.com/julieth_conde61/
Unisinú cada día proyectándose a ser mejores #YoquieroaUnisinu43
https://www.instagram.com/p/CWCaRLirTX0/c/17939126581728764/
1 
comment
17899990799491240
12/11/2021 02:13:58
richard_lopezc
https://www.instagram.com/richard_lopezc/
Muy bueno #YoQuieroaUnisinu81
https://www.instagram.com/p/CWCaRLirTX0/c/178999907994912

04/11/2021 22:04:53
sharolj_01
https://www.instagram.com/sharolj_01/
Que lindos esos recuerdos #YoQuieroaUnisinu39
https://www.instagram.com/p/CV3g_J7ryDn/c/17915214952986708/
1 
comment
17906426669245379
04/11/2021 17:31:54
amador.karo
https://www.instagram.com/amador.karo/
Excelente 👏#yoQuieroaUnisinu31
https://www.instagram.com/p/CV3MAVRrG2Q/c/17906426669245379/
1 
comment
18130992592244152
08/11/2021 17:10:01
julianmorales.007
https://www.instagram.com/julianmorales.007/
Que vale Contaduría
https://www.instagram.com/p/CV3MAVRrG2Q/c/18130992592244152/
0
comment
17971093030460565
10/11/2021 22:52:03
vergarateff
https://www.instagram.com/vergarateff/
Facultad de calidad ❤️❤️🔥🔥 #YoQuieroaUnisinu8
https://www.instagram.com/p/CV3MAVRrG2Q/c/17971093030460565/
0
comment
17994946030387601
12/11/2021 15:29:00
ange1mar
https://www.instagram.com/ange1mar/
👏🏻👏🏻👏🏻 #YoQuieroaUnisinu19
https://www.instagram.com/p/CV3MAVRrG2Q/c/17994946030387601/
1 
comment
17852272400664408
12/11/2021 22:02:09
sof

...

comment
17869381562309727
12/03/2021 20:02:21
david_pca
https://www.instagram.com/david_pca/
Viva la u ,viva nuestra alma mater,#yoquieroAunisinu
https://www.instagram.com/p/CMVARwVrLm7/c/17869381562309727/
1 
comment
17911500052638080
12/03/2021 20:47:46
leila_naranjo
https://www.instagram.com/leila_naranjo/
👏
https://www.instagram.com/p/CMVARwVrLm7/c/17911500052638080/
1 
comment
17951744611422725
12/03/2021 21:27:54
andrex.a.caro
https://www.instagram.com/andrex.a.caro/
Excelente la presencia de la universidad para que sirva de vitrina y asi nuevos estudiantes hagan parte de nuestra querida universidad. #yoquieroaunisinu
https://www.instagram.com/p/CMVARwVrLm7/c/17951744611422725/
1 
comment
17875943936184355
13/03/2021 12:04:03
danyelisnarvaez
https://www.instagram.com/danyelisnarvaez/
Un nuevo centro diagnóstico en la ciudad de Montería requiere doctoras para laborar, interesadas escribir al 3226039968
https://www.instagram.com/p/CMVARwVrLm7/c/17875943936184355/
0
comment
18154627

teresa_guzman_
https://www.instagram.com/teresa_guzman_/
❤️
https://www.instagram.com/p/CMFsi1MHGz1/c/17873930723207633/
1 
comment
17897525794867823
08/03/2021 22:07:20
jose_julian_arcila_diaz
https://www.instagram.com/jose_julian_arcila_diaz/
Excelente 👏👏
https://www.instagram.com/p/CMFsi1MHGz1/c/17897525794867823/
1 
comment
18151024078137283
08/03/2021 22:19:02
david_pca
https://www.instagram.com/david_pca/
Las más felicitaciones a la Dra ,siempre en alto nuestras mujeres de nuestra alma mater #yoquieroAunisinu
https://www.instagram.com/p/CMFsi1MHGz1/c/18151024078137283/
1 
comment
17955084529408201
06/03/2021 02:37:24
danielaacastde
https://www.instagram.com/danielaacastde/
Merecida ese reconocimiento #yoquieroaunisinu ❤️
https://www.instagram.com/p/CMD3JKZnKb4/c/17955084529408201/
1 
comment
18196512229049341
08/03/2021 22:16:25
david_pca
https://www.instagram.com/david_pca/
Siempre con pasos agigantados👏nuestra alma mater #yoquieroAunisinu
https://www.instagram.com/p/CMD3JKZnKb4

comment
18195133327011164
22/02/2021 14:56:34
gisela_0117
https://www.instagram.com/gisela_0117/
Gracias Dios bendiga
https://www.instagram.com/p/CLmFalrru9Y/c/18195133327011164/
2 
comment
17934678763473535
22/02/2021 19:53:13
kathypadronvillalobos
https://www.instagram.com/kathypadronvillalobos/
Hola, como hago para tener el convierto virtual del viernes?
https://www.instagram.com/p/CLmFalrru9Y/c/17934678763473535/
0
reply
17904069844736604
22/02/2021 20:06:39
unisinuoficial
https://www.instagram.com/unisinuoficial/
@kathypadronvillalobos en nuestra página de Facebook esta grabado y lo puedes ver
https://www.instagram.com/p/CLmFalrru9Y/c/17934678763473535/r/17904069844736604/
0
reply
17868334577300533
22/02/2021 20:38:30
kathypadronvillalobos
https://www.instagram.com/kathypadronvillalobos/
@unisinuoficial si, pero lo necesito en físico. Para subirlo en mis redes y darle publicidad en valledupar y Riohacha.
https://www.instagram.com/p/CLmFalrru9Y/c/17934678763473535/r/178683345773005

1 
comment
17882789630045403
29/01/2021 00:40:04
julimcarreazo
https://www.instagram.com/julimcarreazo/
@bh_andrea @andreacastroferia
https://www.instagram.com/p/CKmFaONHFsd/c/17882789630045403/
1 
comment
17994935818313886
29/01/2021 02:54:09
mimamitapsicologa
https://www.instagram.com/mimamitapsicologa/
👏👏👏👏👏
https://www.instagram.com/p/CKmFaONHFsd/c/17994935818313886/
0
comment
17852428463504303
27/01/2021 22:10:44
carlosmaromero
https://www.instagram.com/carlosmaromero/
👏👏👏👏👏👏👏
https://www.instagram.com/p/CKkEgQGHd4m/c/17852428463504303/
1 
comment
17924162995492805
28/01/2021 04:25:28
maffecm17
https://www.instagram.com/maffecm17/
Mmmm
https://www.instagram.com/p/CKkEgQGHd4m/c/17924162995492805/
0
comment
17855388083456358
29/01/2021 20:24:09
jorgeescobaravilez
https://www.instagram.com/jorgeescobaravilez/
Excelente
https://www.instagram.com/p/CKkEgQGHd4m/c/17855388083456358/
0
comment
18193960780059532
29/01/2021 20:24:50
jorgeescobaravilez
https://www.instagram.com/jorgeescobara

vertel0218
https://www.instagram.com/vertel0218/
@vane07v
https://www.instagram.com/p/CJokt1FLv1c/c/17916789448535285/
0
comment
17912434627569084
04/01/2021 20:20:44
mafezalezgon_
https://www.instagram.com/mafezalezgon_/
Alguien sabe si las oficinas de la universidad estan abiertas?
https://www.instagram.com/p/CJokt1FLv1c/c/17912434627569084/
0
reply
17918860003503973
04/01/2021 23:24:29
dilan.ortiz
https://www.instagram.com/dilan.ortiz/
@mafe_zalezgon en la 31 si están atendiendo
https://www.instagram.com/p/CJokt1FLv1c/c/17912434627569084/r/17918860003503973/
0
comment
17851598588467376
04/01/2021 21:33:09
cesarmorelo16
https://www.instagram.com/cesarmorelo16/
@estefav.a
https://www.instagram.com/p/CJokt1FLv1c/c/17851598588467376/
2 
comment
17931290617462672
04/01/2021 22:02:04
almaisabel07
https://www.instagram.com/almaisabel07/
@unisinuoficial Que horario de atención tienen?
https://www.instagram.com/p/CJokt1FLv1c/c/17931290617462672/
1 
comment
18184957375013370
02/01/2021 12:39:

In [30]:
resultado = crear_dataframe(comments_instagram)
resultado

,type,id,create_time,user,url_user,comment,url_comment,likes
0,comment,18155725081175986,19/11/2021 00:42:56,13marlis,https://www.instagram.com/13marlis/,#YoquieroaUnisinu97 👏👏👏,https://www.instagram.com/p/CWb977_rDLA/c/1815...,0
1,comment,17877329669594065,19/11/2021 00:48:32,amador.karo,https://www.instagram.com/amador.karo/,👏#yoQuieroaUnisinu31,https://www.instagram.com/p/CWb977_rDLA/c/1787...,0
2,comment,17912870729162297,19/11/2021 01:06:06,leila_naranjo,https://www.instagram.com/leila_naranjo/,👏,https://www.instagram.com/p/CWb977_rDLA/c/1791...,0
3,comment,18152281534200727,19/11/2021 01:51:36,keila_vergaraa,https://www.instagram.com/keila_vergaraa/,#YoQuieroAUnisinu98,https://www.instagram.com/p/CWb977_rDLA/c/1815...,0
4,comment,17898428138420919,19/11/2021 02:05:38,brayan.vinasco,https://www.instagram.com/brayan.vinasco/,"Frente al poder reptan, son incapaces de estab...",https://www.instagram.com/p/CWb977_rDLA/c/1789...,0
...,...,...,...,...,...,...,...,...
2121,reply,18067802305252077,08/01/2021 15:09:55,jarguk,https://www.instagram.com/jarguk/,@miguegarciaprasca_ nada,https://www.instagram.com/p/CJdv0pvrXYd/c/1818...,0
2122,comment,17866861595272443,26/01/2021 15:49:42,shareth_25,https://www.instagram.com/shareth_25/,Me pueden hacer el favor de dar información de...,https://www.instagram.com/p/CJdv0pvrXYd/c/1786...,1
2123,reply,17950575982407697,26/01/2021 16:18:59,unisinuoficial,https://www.instagram.com/unisinuoficial/,@shareth_25 regalame tu numero para comunicarn...,https://www.instagram.com/p/CJdv0pvrXYd/c/1786...,0
2124,comment,18090904723237021,30/01/2021 20:20:38,maserrato_16,https://www.instagram.com/maserrato_16/,Me pueden dar información de GERENCIA Y DIRECC...,https://www.instagram.com/p/CJdv0pvrXYd/c/1809...,0


In [28]:
exportar(resultado)